1. data [x]
    - get the vrts for esa,ldar,s1,edem [x]
    - get the tifs from tls and rng [x]

2. datapre [x]
    - clip by bbox [x] 
    - split train from test (cpu modelling continual ml?) [x]

3. datamod [tab][+]
    - step 1: predit ldem [x]
    - step 2: get scores :
        -    with residual [x]
        -  without residual [+]
    - step 3: style transfer [x]: not impactfull at this scale. maybe at step 4
    - step 4: expand prediction range by N pixels same model [-]
    - step 5: data fusion with ldem [-]
4. datamod [img]

In [1]:
import os
from glob import glob
from daug import daug_isolate_data_by_bbox

In [2]:
outdir4749 =  "/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/reproj4749"  
ldem_vrt = f"{outdir4749}/ldem.vrt"
cmd = f"gdalbuildvrt -overwrite {ldem_vrt} {outdir4749}/*.tif"
os.system(cmd)

esawc_vrt = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRT/esawc/esawc.vrt"
s1_vrt = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRT/s1/s1.vrt"
edem_egm_vrt = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRT/edem_egm/edem_egm.vrt" 

tls_fn = "/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/reproj4749/AngkorDTM50cm.tif"
rng_files = glob("/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/reproj4749/*.tif")
rgn_files = [f for f in rng_files if "AngkorDTM50cm" not in f and "delta_s_m2070" not in f]

vrt_files = [esawc_vrt, s1_vrt,edem_egm_vrt, ldem_vrt] # these are the vrt files 
temp_dir = 'temp'
os.makedirs(temp_dir, exist_ok=True)

0...10...20...30...40...50...60...70...80...90...100 - done.


In [3]:
ylist,nlist = daug_isolate_data_by_bbox(tls_fn, vrt_files, temp_dir)

Reference GeoTIFF: /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/reproj4749/AngkorDTM50cm.tif
  Bounding box: BoundingBox(left=103.73573115613353, bottom=13.317802380912203, right=103.99762004502242, top=13.47535793646776)
  Transform: | 0.00, 0.00, 103.74|
| 0.00,-0.00, 13.48|
| 0.00, 0.00, 1.00|
  Width: 2357, Height: 1418

Processing /media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRT/esawc/esawc.vrt...
  Saved aligned data to temp/aligned_esawc.tif

Processing /media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRT/s1/s1.vrt...
  Saved aligned data to temp/aligned_s1.tif

Processing /media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRT/edem_egm/edem_egm.vrt...
  Saved aligned data to temp/aligned_edem_egm.tif

Processing /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/reproj4749/ldem.vrt...
  Saved aligned data to temp/aligned_ldem.tif


In [5]:
topoxcale_dir = "/home/ljp238/Documents/UoE/topoxcale/"
import sys 
sys.path.append(topoxcale_dir)
from topoxcale.sagaxcale import gwrdownxcale

import numpy as np
import rasterio
import logging
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import root_mean_squared_error, r2_score

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# --- Step 1: Helper Functions --- 

def read_dem(fpath, lval, hval, band=1):
    """Reads DEM from GeoTIFF, applies valid data mask, and filters values based on provided thresholds."""
    logger.info(f"Reading DEM from: {fpath}")
    try:
        with rasterio.open(fpath) as src:
            data = src.read(band).astype(np.float32)  # Read specific band
            ndv = src.nodata

            # Build the valid data mask
            mask = np.ones_like(data, dtype=bool)
            if ndv is not None:
                mask &= (data != ndv)
            mask &= (data > lval) & (data < hval)

            # Apply mask: set invalid pixels to np.nan
            filtered_data = np.where(mask, data, np.nan)
            return filtered_data, mask, src
    except Exception as e:
        logger.error(f"Error reading DEM: {e}")
        raise

def read_geotiff(file_path):
    """Reads a GeoTIFF file and returns its data and dataset."""
    logger.info(f"Reading GeoTIFF: {file_path}")
    try:
        with rasterio.open(file_path) as dataset:
            data = dataset.read()
            return data, dataset
    except Exception as e:
        logger.error(f"Error reading GeoTIFF: {e}")
        raise

def apply_mask(data, mask, band=0):
    """Applies a mask to a dataset (multi-band) to exclude invalid pixels."""
    logger.info("Applying mask to data")
    return np.where(mask, data[band] if data.ndim == 3 else data, np.nan)

def save_geotiff(output_path, array, reference_src, nodata_value=None):
    """Saves a numpy array as a GeoTIFF using spatial properties from a reference raster."""
    logger.info(f"Saving GeoTIFF to: {output_path}")
    try:
        if nodata_value is not None:
            array = np.nan_to_num(array, nan=nodata_value)

        with rasterio.open(
            output_path,
            'w',
            driver='GTiff',
            height=array.shape[0],
            width=array.shape[1],
            count=1,
            dtype=array.dtype,
            crs=reference_src.crs,
            transform=reference_src.transform,
            nodata=nodata_value
        ) as dst:
            dst.write(array, 1)  # Write to first band
    except Exception as e:
        logger.error(f"Error saving GeoTIFF: {e}")
        raise

# --- Step 2: Main Code Execution ---

# File paths and threshold values
lval, hval = -99, 9000

# Read datasets
ldem, mask, ldem_src = read_dem('temp/aligned_ldem.tif', lval=lval, hval=hval)
edem, edem_src = read_geotiff('temp/aligned_edem_egm.tif')
s1, s1_src = read_geotiff('temp/aligned_s1.tif')
esa, esa_src = read_geotiff('temp/aligned_esawc.tif')

# Apply masks to each dataset
edem_masked = apply_mask(edem, mask)
esa_masked = apply_mask(esa, mask)
s1_vv_masked = apply_mask(s1, mask, band=0)
s1_vh_masked = apply_mask(s1, mask, band=1)
ldem_masked = np.where(mask, ldem, np.nan)

# Compute difference
zdif_masked = edem_masked - ldem_masked

# Stack input features
X_stack = np.stack([edem_masked, esa_masked, s1_vv_masked, s1_vh_masked], axis=0)

# Flatten features and targets
X_flat = X_stack.reshape(4, -1).T  # (n_samples, 4)
Y1_flat = ldem_masked.flatten()    # (n_samples,)
Y2_flat = zdif_masked.flatten()

# Keep only valid (non-NaN) samples
valid_mask = ~np.isnan(X_flat).any(axis=1) & ~np.isnan(Y1_flat) & ~np.isnan(Y2_flat)
X_final = X_flat[valid_mask]
Y_final = np.stack([Y1_flat[valid_mask], Y2_flat[valid_mask]], axis=1)  # (n_samples, 2)

# Split into train/test (90% train, 10% test)
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size=0.1, random_state=42, shuffle=True)

# --- Step 3: Train CatBoost Model (Multi-target) ---
logger.info("Training CatBoost model...")
model = CatBoostRegressor(
    iterations=1000,
    #learning_rate=0.05,
    #depth=6,
    loss_function='MultiRMSE',
    verbose=100
)
model.fit(X_train, Y_train)

# --- Step 4: Evaluate Model ---
logger.info("Evaluating model...")
Y_pred = model.predict(X_test)

metrics = {}
for i, target_name in enumerate(['LDEM', 'ZDIFF']):
    rmse = root_mean_squared_error(Y_test[:, i], Y_pred[:, i])
    r2 = r2_score(Y_test[:, i], Y_pred[:, i])
    metrics[target_name] = {'RMSE': rmse, 'R2': r2}

# Print evaluation results
for target, scores in metrics.items():
    logger.info(f"--- {target} ---")
    logger.info(f"RMSE: {scores['RMSE']:.4f}")
    logger.info(f"R²:   {scores['R2']:.4f}")

# --- Step 5: Predict Full Image (using Unmasked Features) ---
logger.info("Predicting full image...")
edem_unmasked = edem[0] if edem.ndim == 3 else edem
esa_unmasked = esa[0] if esa.ndim == 3 else esa
s1_vv_unmasked = s1[0]
s1_vh_unmasked = s1[1]

# Stack full feature arrays
X_full_stack = np.stack([edem_unmasked, esa_unmasked, s1_vv_unmasked, s1_vh_unmasked], axis=0)

# Flatten for prediction
X_full_flat = X_full_stack.reshape(4, -1).T  # (n_pixels, 4)

# Predict
Y_full_pred = model.predict(X_full_flat)

# Reshape back to image
H, W = edem_unmasked.shape
Y1_pred_map = Y_full_pred[:, 0].reshape(H, W)  # Predicted LDEM
Y2_pred_map = Y_full_pred[:, 1].reshape(H, W)  # Predicted ZDIFF

# Prepare final prediction images
pred_di = np.nan_to_num(Y1_pred_map, nan=-9999)
pred_id = np.nan_to_num(edem_unmasked - Y2_pred_map, nan=-9999)

# Save predicted outputs as GeoTIFF
save_geotiff('temp/predicted_di.tif', pred_di, edem_src, nodata_value=-9999)
save_geotiff('temp/predicted_id.tif', pred_id, edem_src, nodata_value=-9999)

# Optional postprocessing
postprocessing = False
if postprocessing:
    xpath = "temp/aligned_edem_egm.tif"
    ypath = "temp/predicted_di.tif"
    opath = "temp/predicted_di_wr.tif"
    gwrdownxcale(xpath, ypath, opath, oaux=False, epsg_code=4979, clean=True)

logger.info("Processing complete.")


2025-04-29 02:46:11,627 - INFO - Reading DEM from: temp/aligned_ldem.tif
2025-04-29 02:46:11,679 - INFO - Reading GeoTIFF: temp/aligned_edem_egm.tif


2025-04-29 02:46:11,706 - INFO - Reading GeoTIFF: temp/aligned_s1.tif
2025-04-29 02:46:11,767 - INFO - Reading GeoTIFF: temp/aligned_esawc.tif
2025-04-29 02:46:11,774 - INFO - Applying mask to data
2025-04-29 02:46:11,779 - INFO - Applying mask to data
2025-04-29 02:46:11,793 - INFO - Applying mask to data
2025-04-29 02:46:11,812 - INFO - Applying mask to data
2025-04-29 02:46:12,240 - INFO - Training CatBoost model...


0:	learn: 7.0165000	total: 85.2ms	remaining: 1m 25s
100:	learn: 2.7999378	total: 5.9s	remaining: 52.5s
200:	learn: 2.6948879	total: 11.2s	remaining: 44.5s
300:	learn: 2.6604480	total: 16.6s	remaining: 38.5s
400:	learn: 2.6354505	total: 22s	remaining: 32.9s
500:	learn: 2.6191455	total: 27.5s	remaining: 27.4s
600:	learn: 2.6051187	total: 33.1s	remaining: 22s
700:	learn: 2.5943860	total: 38.5s	remaining: 16.4s
800:	learn: 2.5845746	total: 44.1s	remaining: 11s
900:	learn: 2.5752458	total: 49.7s	remaining: 5.46s


2025-04-29 02:47:11,567 - INFO - Evaluating model...


999:	learn: 2.5673689	total: 55.3s	remaining: 0us


2025-04-29 02:47:11,841 - INFO - --- LDEM ---
2025-04-29 02:47:11,842 - INFO - RMSE: 1.8637
2025-04-29 02:47:11,843 - INFO - R²:   0.8858
2025-04-29 02:47:11,844 - INFO - --- ZDIFF ---
2025-04-29 02:47:11,845 - INFO - RMSE: 1.8040
2025-04-29 02:47:11,846 - INFO - R²:   0.8467
2025-04-29 02:47:11,847 - INFO - Predicting full image...
2025-04-29 02:47:12,811 - INFO - Saving GeoTIFF to: temp/predicted_di.tif
2025-04-29 02:47:12,879 - INFO - Saving GeoTIFF to: temp/predicted_id.tif
2025-04-29 02:47:12,934 - INFO - Processing complete.


In [53]:
# style transfer here @GWR, ML [ready this functions]
### do from topoxcale GWR for now []
# merging/blending [ml version]

do this pipeline with image models??

In [6]:
# give ldem,predicted, dsm pick min pixel across all 3 dems per pixels to build final product
# thresholds apply , and the orther apply like listed above